In [1]:
import mlflow
import mlflow.pyfunc
import mlflow.pytorch
from  mlflow.tracking import MlflowClient
import os
import numpy as np
import torch
from torchvision import datasets, transforms
import pandas as pd
from PIL import Image
import base64
import io

In [2]:
experiment_name = "pytorch_exp_1"
tracking_uri = os.environ.get("TRACKING_URL")
client = MlflowClient(tracking_uri=tracking_uri)
mlflow.set_tracking_uri(tracking_uri)
experiments = client.list_experiments()

experiment_names = []
for exp in experiments:
    experiment_names.append(exp.name)
if experiment_name not in experiment_names:
    try:
        mlflow.create_experiment(experiment_name)
    except:
        pass
mlflow.set_experiment(experiment_name)

<Experiment: artifact_location='file:///home/jovyan/tractrain2022/notebooks/part-1/deployment/mlruns/1', experiment_id='1', lifecycle_stage='active', name='pytorch_exp_1', tags={}>

### Download mnist png files from https://github.com/myleott/mnist_png

```
cd ~/
git clone https://github.com/myleott/mnist_png
cd mnist_png
tar xvzf mnist_png.tar.gz
```

In [28]:
import os
from glob import glob
_path = "/home/jovyan/mnist_png/mnist_png/testing/"
file_list = [y for x in os.walk(_path) for y in glob(os.path.join(x[0], '*.png'))]

In [29]:
test_size = min(len(file_list),100)
file_list = file_list[:test_size]
encoded_string_list = []
for _file in file_list:
    with open(_file, "rb") as image_file:
        encoded_string = base64.b64encode(image_file.read()).decode('utf-8')
    encoded_string_list.append(encoded_string)
input_df = pd.DataFrame(encoded_string_list,columns=["images"])
print(test_size)

100


In [40]:
print(len(input_df))
run_id = "1d7df19693dd4b8d94554b7aacce4caf"
with mlflow.start_run(run_id=run_id):
    loaded_model = mlflow.pyfunc.load_model(mlflow.get_artifact_uri("pytorch-model"))

    # Make a few predictions
    predictions = loaded_model.predict(input_df)
predictions

100
Pytorch model initialized
Predicting 100 samples


array([2, 2, 2, 2, 2, 2, 2, 8, 2, 0, 2, 8, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2,
       2, 2, 2, 7, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 1, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 6, 2, 2, 2])

### Sample code to test REST API Endpoint Deployment

In [41]:
import requests
url = "http://127.0.0.1:5000/invocations"
data_json = input_df.to_json(orient="split",index=False)
headers = {"Content-Type":"application/json"}
response = requests.post(url,data=data_json,headers=headers)
if response.status_code == 200:
    print(response.json())
else:
    print("REST API deployment is in progress -- please try again in a few minutes!")

[2, 2, 2, 2, 2, 2, 2, 8, 2, 0, 2, 8, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 7, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 6, 2, 2, 2]
